A script to prototype supervoxel extraction of the Keller data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pyqtgraph as pg
import pyspark

from janelia_core.cell_extraction.super_voxels import extract_super_voxels_in_brain
from janelia_core.dataprocessing.baseline import percentile_filter_multi_d
from janelia_core.fileio.exp_reader import find_images
from janelia_core.fileio.exp_reader import read_img_file

In [3]:
%matplotlib qt

## Parameters go here

In [5]:
# Parameters for where the images are located
image_base_folder = r'K:\\SV4'
image_main_folder = 'CW_17-08-23'
image_sub_folder = 'L4-561nm-ROIMonitoring_20170823_193101.corrected'
image_processed_folder = 'Results\\WeightFused'
image_ext = r'weightFused.TimeRegistration.templateSpace.klb'

# Parameters for where the brain mask is located
brain_mask_folder = r'A:\\projects\\keller_vnc\\data\\registered_brain_mask'
brain_mask_file = 'thresholdedVoxelMapSmooth.klb'

# Parameters for how we extract supervoxels
voxel_size_per_dim = [1, 5, 5]
brain_mask_perc = .75 # Percentage of a super voxel which must be in the brain mask to be processed

# Parameters for baseline extraction
window_length = 1201
filter_start = -600
write_offset = 600

## Create a pyspark context

In [6]:
conf = pyspark.SparkConf().setMaster('local[20]').setAll([
    ('spark.executor.memory', '10g'), ('spark.driver.memory','400g'), ('spark.driver.maxResultSize', '300g')])
sc = pyspark.SparkContext(conf=conf)

## Find the images

In [7]:
image_folder = Path(image_base_folder) / image_main_folder / image_sub_folder / image_processed_folder
imgs = find_images(image_folder=image_folder, image_ext=image_ext, image_folder_depth=1)

Searching for image files...
Found 7208 images.


## Load the brain mask

In [8]:
brain_mask_file = Path(brain_mask_folder) / brain_mask_file

In [9]:
brain_mask = read_img_file(brain_mask_file)

## Extract super voxels

In [1]:
extract_t0 = time()
roi_vls, rois = extract_super_voxels_in_brain(images=imgs, voxel_size_per_dim=voxel_size_per_dim, brain_mask=brain_mask,
                                              brain_mask_perc=brain_mask_perc, sc=sc)
extract_t1 = time()

print('ROI Extraction time: ' + str(extract_t1 - extract_t0))

NameError: name 'time' is not defined

## Calculate Baselines

In [ ]:
baseline_t0 = time()
baseline = percentile_filter_multi_d(roi_vls, window_length=window_length, filter_start=filter_start, write_offset=write_offset, 
                                     p = .01, n_processes=80)
baseline_t1 = time()
print('Baseline calculation time: ' + str(baseline_t1 - baseline_t0))

## Calculate \Delta F/F

In [ ]:
dff_t0 = time()
dff = roi_vls/(baseline + .01)
dff_t1 = time()
print('DFF calculation time: ' + str(dff_t1 - dff_t0))

In [ ]:
plt.plot(roi_vls[:,100])
plt.plot(baseline[:,100])

In [ ]:
roi_vls.shape

In [ ]:
np.mean(baseline)

In [ ]:
1/2